# Bước 0:

In [2]:
import pandas as pd
url = "https://raw.githubusercontent.com/huongd17at089/income_prediction/master/data_adult.csv"
df = pd.read_csv(url)

In [45]:
df.head(5)

,Unnamed: 0,X,age,workclass,education,educational.num,marital.status,race,gender,hours.per.week,income
0,1,1,25,Private,11th,7,Never-married,Black,Male,40,<=50K
1,2,2,38,Private,HS-grad,9,Married-civ-spouse,White,Male,50,<=50K
2,3,3,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,White,Male,40,>50K
3,4,4,44,Private,Some-college,10,Married-civ-spouse,Black,Male,40,>50K
4,5,5,34,Private,10th,6,Never-married,White,Male,30,<=50K


In [46]:
df.tail(5)

,Unnamed: 0,X,age,workclass,education,educational.num,marital.status,race,gender,hours.per.week,income
46028,46029,46029,27,Private,Assoc-acdm,12,Married-civ-spouse,White,Female,38,<=50K
46029,46030,46030,40,Private,HS-grad,9,Married-civ-spouse,White,Male,40,>50K
46030,46031,46031,58,Private,HS-grad,9,Widowed,White,Female,40,<=50K
46031,46032,46032,22,Private,HS-grad,9,Never-married,White,Male,20,<=50K
46032,46033,46033,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,White,Female,40,>50K


In [18]:
df.drop(["X", "Unnamed: 0"], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46033 entries, 0 to 46032
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       46033 non-null  int64 
 1   X                46033 non-null  int64 
 2   age              46033 non-null  int64 
 3   workclass        46033 non-null  object
 4   education        46033 non-null  object
 5   educational.num  46033 non-null  int64 
 6   marital.status   46033 non-null  object
 7   race             46033 non-null  object
 8   gender           46033 non-null  object
 9   hours.per.week   46033 non-null  int64 
 10  income           46033 non-null  object
dtypes: int64(5), object(6)
memory usage: 3.9+ MB


In [20]:
print(any(df.duplicated().tolist()))

False


In [6]:
continuous_vars = ["age", "educational.num", "hours.per.week"]
categorical_vars = ["workclass", "education", "marital.status", "race", "gender"]
target = "income"

In [9]:
df1 = df[continuous_vars]
df1.sample(5)

,age,educational.num,hours.per.week
34044,40,14,40
18799,21,10,40
22313,24,12,40
1632,26,9,25
43568,49,13,40


In [12]:
df1.describe()

,age,educational.num,hours.per.week
count,46033.000000,46033.000000,46033.000000
mean,38.556601,10.128929,40.948407
std,13.204268,2.564064,12.011069
min,17.000000,1.000000,1.000000
25%,28.000000,9.000000,40.000000
50%,37.000000,10.000000,40.000000
75%,47.000000,13.000000,45.000000
max,90.000000,16.000000,99.000000


In [15]:
df2 = df[categorical_vars]
df2.sample(5)

,workclass,education,marital.status,race,gender
14042,Self-emp-not-inc,HS-grad,Married-civ-spouse,White,Male
16252,Self-emp-inc,Bachelors,Married-civ-spouse,White,Male
894,Private,HS-grad,Separated,White,Male
9664,Private,HS-grad,Married-civ-spouse,White,Male
38496,Private,Preschool,Married-civ-spouse,White,Male


In [44]:
for var in categorical_vars:
  info = pd.DataFrame()
  info["Frequency"] = df[var].value_counts()
  info["Percentage"] = info["Frequency"]/df2.shape[0]*100
  print(info)
  print(info[info["Frequency"] == max(info["Frequency"])])
  print("-----------------------------------------------------------")


                  Frequency  Percentage
Private               33906   73.655856
Self-emp-not-inc       3862    8.389634
Local-gov              3136    6.812504
State-gov              1981    4.303434
Self-emp-inc           1695    3.682141
Federal-gov            1432    3.110812
Without-pay              21    0.045619
         Frequency  Percentage
Private      33906   73.655856
-----------------------------------------------------------
              Frequency  Percentage
HS-grad           14972   32.524493
Some-college      10036   21.801751
Bachelors          7772   16.883540
Masters            2590    5.626398
Assoc-voc          1978    4.296917
11th               1631    3.543110
Assoc-acdm         1529    3.321530
10th               1239    2.691547
7th-8th             844    1.833467
Prof-school         810    1.759607
9th                 687    1.492408
12th                599    1.301240
Doctorate           576    1.251276
5th-6th             468    1.016662
1st-4th           